# Covid detection using yolov5

In [ ]:
import pandas as pd
import numpy as np
import os
import ast
import shutil
import glob
import cv2 as cv
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
train_path="../input/siim-covid19-resized-to-256px-jpg/train"
BATCH_SIZE=32
EPOCHS=50
IMG_SIZE=256

In [ ]:
df=pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
df['image_label']=df.label.apply(lambda x: x.split(" ")[0])
df['id']=df['id'].apply(lambda x: x.split("_")[0])
df['filepath']=df['id'].apply(lambda x: train_path+'/' +x+'.jpg')
df.head()

In [ ]:
#Trying a sample image
plt.imshow(plt.imread(df.filepath[0]))

In [ ]:
%cd ../
os.makedirs('/kaggle/tmp',exist_ok=True)

In [ ]:
%cd tmp

In [ ]:
meta_df=pd.read_csv("../input/siim-covid19-resized-to-256px-jpg/meta.csv")
train_df=meta_df[meta_df.split=='train']
train_df.drop(columns=['split'],inplace=True)
train_df.columns=['id','dim0','dim1']
train_df.head()

In [ ]:
df=df.merge(train_df,on='id',how='left')
df.head(5)

# Train validation split

In [ ]:
df_train,df_valid=train_test_split(df,test_size=0.3,random_state=42, stratify=df.image_label.values)
df_train.loc[:,"split"]='train'
df_valid.loc[:,"split"]='valid'

df=pd.concat([df_train,df_valid]).reset_index(drop=True)

df.head()

In [ ]:
pwd

In [ ]:
os.makedirs('covid/images/train',exist_ok=True)
os.makedirs('covid/images/valid',exist_ok=True)

os.makedirs('covid/labels/train',exist_ok=True)
os.makedirs('covid/labels/valid',exist_ok=True)

In [ ]:
ls covid/images/

In [ ]:
pwd

In [ ]:
df.head()

In [ ]:
for i in tqdm(range(len(df))):
    x=df.loc[i]
    if x.split=="train":
        shutil.copyfile(f'../input/siim-covid19-resized-to-256px-jpg/train/{x.id}.jpg',f'/kaggle/tmp/covid/images/train/{x.id}.jpg')
    else:
         shutil.copyfile(f'../input/siim-covid19-resized-to-256px-jpg/train/{x.id}.jpg',f'/kaggle/tmp/covid/images/valid/{x.id}.jpg')
        

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# Install dependencies
%pip install -qr requirements.txt  # install dependencies

In [ ]:
%cd /kaggle/tmp

In [ ]:
import yaml
data_yaml=dict(
train='/kaggle/tmp/covid/images/train',
val='/kaggle/tmp/covid/images/valid',
nc=2,
names=['none','opacity'])

with open('/kaggle/tmp/yolov5/data/data.yaml','w') as outfile:
    yaml.dump(data_yaml,outfile,default_flow_style=True)